In [1]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00


In [2]:
import pandas as pd

In [3]:
import pymysql

# replace the values in this connection string with the appropriate values for your database
conn = pymysql.connect(
    host='',
    user='',
    password='',
    db='',
    charset='',
    cursorclass=pymysql.cursors.DictCursor
)

## Exploratory Question 1 

Purpose of Query - I want to see what distinct years the API was able to scrape from the website and what era of data I am dealing with

In [4]:
cursor = conn.cursor()
cursor.execute(
    'SELECT DISTINCT(YEAR(game_date)) AS stat_years FROM player_stats ps ORDER BY game_date DESC;'
    )
results = cursor.fetchall()
df = pd.DataFrame(results)
df

,stat_years
0,2013
1,2012
2,2011
3,2010
4,2009
5,2008
6,2007
7,2006
8,2004
9,1977


Insight - Majority of the stats are from the 2004-2013 era of the NBA meaning a lot of the players from the current NBA will not be overlapping. This also means that it is important to understand the evolution of the league and how the game was played considering it has changed greatly over the past 10-20 years.

## Exploratory Question 2

Purpose of Query - I want to see the breakdown of the players from the NBA player API to see if there are any missing values or an overflow in one specific position

In [6]:
cursor = conn.cursor()
cursor.execute(
    'SELECT position, COUNT(*) AS player_count FROM players p GROUP BY `position` ;'
    )
results = cursor.fetchall()
df = pd.DataFrame(results)
df

,position,player_count
0,C,14
1,G,54
2,F,40
3,C-F,1
4,F-C,2
5,G-F,1
6,,2487
7,F-G,1


Insight - Out of the 2600 players that were web scraped from the API, a majority of them have no position assigned to them aside from a select few. This means that aside from the essentials of the table (id, name, team), I need to be careful querying on other columns in the players table since there is a good chance for missing values

## Exploratory Question 3

Purpose of Query - I want to explore the diversity of the game logs that were pulled from the player stats API

In [8]:
cursor = conn.cursor()
cursor.execute(
    'WITH player_count AS (SELECT player_name, COUNT(*) AS repeating_count FROM player_stats ps GROUP BY player_name) SELECT player_name, repeating_count FROM player_count WHERE repeating_count > 9 ORDER BY repeating_count DESC;'
    )
results = cursor.fetchall()
df = pd.DataFrame(results)
df

,player_name,repeating_count
0,Thabo Sefolosha,14
1,Keith Bogans,13
2,Andray Blatche,13
3,Tyrus Thomas,12
4,James Jones,10
5,Tony Parker,10
6,Andrei Kirilenko,10
7,Shawn Marion,10
8,Kevin Martin,10
9,Drew Gooden,10


Insight - The results show that only 12 players have double digit occurrences of repeats, which should indicate that of the 2,575 records pulled from the player stats API, very few are the same players' information being reguritated. This should mean that the player sample size is vast and significant.

## Exploratory Question 4

Purpose of Query - I want to see how many teams from the data from the 1940s are no longer in existence anymore

In [10]:
cursor = conn.cursor()
cursor.execute(
    'SELECT COUNT(*) AS changed_teams FROM teams t WHERE LENGTH(city) < 2;'
    )
results = cursor.fetchall()
df = pd.DataFrame(results)
df

,changed_teams
0,15


Insight - Since there are 30 teams currently in the association, this means that player data from these 15 "old" teams has to be disregarded since they do not apply to teams in existence currently, thus making it easier to query the tables once joined.

## Exploratory Question 5

Purpose of Query - I want to see the players have duplicates in this data pulled from the Basketball Reference website (not an API or web scrape)

In [5]:
cursor = conn.cursor()
cursor.execute(
    'WITH player_count AS (SELECT Player , COUNT(Player) AS moved_player FROM recent_season rs GROUP BY Player) SELECT Player, moved_player FROM player_count WHERE moved_player > 1 GROUP BY Player ;'
    )
results = cursor.fetchall()
df = pd.DataFrame(results)
df

,Player,moved_player
0,Nickeil Alexander-Walker,3
1,Ryan Arcidiacono,3
2,Mo Bamba,3
3,Will Barton,3
4,Darius Bazley,3
5,Malik Beasley,3
6,Patrick Beverley,3
7,Saddiq Bey,3
8,Goga Bitadze,3
9,Jamaree Bouyea,3


Insight - This displays that 55 total players have duplicate data in the table,which is possibly a result of being traded or cut/signed multiple times throughout the season. Regardless, this means that I must apply filters on my future queries that specify to eliminate duplicates before trying to delve into future insights

## Primary Question



*   Question - Has increased overall scoring over the past 2 decades affected the quality of defense by the league's best scorers?
*   Business Justification - This can be used to influence roster decision making in the coming years, deciding whether investments in high level defenders who are limited offensively are needed as much as previous years.
*   SQL features used - GROUP BY, VIEW, CTE, JOIN



In [4]:
cursor = conn.cursor()
cursor.execute(
    'SELECT cd.player_name , cd.points , cd.stocks , AVG(phs.stocks) AS past_stock_average , AVG(nhs.stocks) AS new_stock_average FROM combined_data cd  CROSS JOIN past_high_scorers phs CROSS JOIN new_high_scorers nhs GROUP BY cd.player_name ;'
    )
results = cursor.fetchall()
df = pd.DataFrame(results)
df

,player_name,points,stocks,past_stock_average,new_stock_average
0,Tracy McGrady,30.0,4.0,1.9531,1.8
1,Chris Crawford,27.0,1.0,1.9531,1.8
2,Shaquille O'Neal,29.0,5.0,1.9531,1.8
3,Tim Duncan,31.0,8.0,1.9531,1.8
4,Shawn Marion,31.0,3.0,1.9531,1.8
...,...,...,...,...,...
76,Shai Gilgeous-Alexander,31.4,2.6,1.9531,1.8
77,Damian Lillard,32.2,1.2,1.9531,1.8
78,Lauri Markkanen,25.6,1.2,1.9531,1.8
79,Donovan Mitchell,28.3,1.9,1.9531,1.8


**Related SQL**

CREATE VIEW combined_data AS (
	WITH past_stats AS (SELECT player_name , points , steals , blocks , minutes FROM player_stats ps WHERE points > 25 GROUP BY player_name), 
	current_stats AS (SELECT Player, PTS, STL, BLK FROM recent_season rs WHERE PTS > 25 GROUP BY Player) 
	SELECT ps.player_name, ps.points , SUM(ps.steals+ps.blocks) AS stocks 
	FROM past_stats ps 
	GROUP BY player_name  
	UNION 
	SELECT cs.player, cs.pts AS points, SUM(cs.stl + cs.blk) AS new_stocks 
	FROM current_stats cs 
	GROUP BY player
);

CREATE VIEW past_high_scorers AS (
	WITH past_stats AS (SELECT player_name , points , steals , blocks , minutes FROM player_stats ps WHERE points > 25 GROUP BY player_name)
	SELECT ps.player_name, ps.points , SUM(ps.steals+ps.blocks) AS stocks 
	FROM past_stats ps 
	GROUP BY player_name
);

CREATE VIEW new_high_scorers AS (
	WITH current_stats AS (SELECT Player, PTS, STL, BLK FROM recent_season rs WHERE PTS > 25 GROUP BY Player)
	SELECT cs.player, cs.pts AS points, SUM(cs.stl + cs.blk) AS stocks
	FROM current_stats cs
	GROUP BY player
);

Insight - This demonstrates the evolution of defense by the top scorers over the past 10-20 years and how the "stocks" (steals + blocks combined) have fallen by nearly 0.2 by these elite players, which is a significant number. An actionable recommendation based on this is that high level defenders are needed more than ever on rosters with these top end scoring talents in order to mitigate the overall decline in their defensive production.

## Related to Primary Question 1



*   Question - How many players drafted between 2004 and 2013 are still capable of being productive starting players in the league today?
*   Business Justification - Since many top players have declined defensively as a result of an added burden to score, there is need for players to be able to physically exert more energy defensively which has historically been a tough thing for older players to do. I want to see how many players drafted 10+ years ago are currently capable and if subsequently there needs to be a youth movement league wide.
*   SQL features used - CASE, Window Function, JOIN



In [20]:
cursor = conn.cursor()
cursor.execute(
    "SELECT DISTINCT(ps.player_name) ,ps.team_name , rs.tm AS new_team, CASE	WHEN rs.MP > 30 THEN 'Yes' ELSE 'No' END AS starter_level, RANK() OVER(ORDER BY rs.MP DESC) AS player_minutes_rank FROM player_stats ps LEFT JOIN recent_season rs ON ps.player_name = rs.Player WHERE rs.tm IS NOT NULL AND rs.tm <> 'TOT' AND rs.MP > 20 ORDER BY starter_level;"
    )
results = cursor.fetchall()
df = pd.DataFrame(results)
df

,player_name,team_name,new_team,starter_level,player_minutes_rank
0,Nicolas Batum,Portland Trail Blazers,LAC,No,109
1,Alec Burks,Utah Jazz,DET,No,107
2,Khris Middleton,Detroit Pistons,MIL,No,106
3,Reggie Jackson,Oklahoma City Thunder,LAC,No,101
4,Marcus Morris,Houston Rockets,LAC,No,100
5,Mike Conley,Memphis Grizzlies,UTA,No,94
6,Harrison Barnes,Golden State Warriors,SAC,Yes,60
7,Jrue Holiday,Philadelphia 76ers,MIL,Yes,55
8,Chris Paul,LA Clippers,PHO,Yes,63
9,Chris Paul,New Orleans Pelicans,PHO,Yes,63


Insight - These results are ranked by their league ranking in minutes across all players this past 2022-23 season. This shows that there are only 33 players out of the possible 450 that are currently playing meaningful minutes and of these 33, 27 of them are starting caliber players. This displays the shift to a youth movement by the league due to the need for fresher legs on the defensive end on a night-to-night basis.

## Related to Primary Question 2



*   Question - For player performances where they play high volume of minutes, what is the average differential between possessions they are responsible for losing (turnovers) and possessions they are responsible for saving (stocks)?
*   Business Justification - This is imperative to figuring out if these high volume players are able to remaining a positive on the possession battle throughout the flow of the game and if their offense is enough to make up for lacking defense in comparison to those playing less minutes.
*   SQL features used - CTE, Subquery



In [21]:
cursor = conn.cursor()
cursor.execute(
    "WITH high_minute_stats AS (SELECT player_name , points, (steals+blocks) AS stocks, turnovers, ((steals+blocks) - turnovers) AS net_possession_differential FROM player_stats ps  WHERE minutes IN (SELECT minutes FROM player_stats ps WHERE minutes > 40)) SELECT AVG(points) AS average_points, AVG(net_possession_differential) AS average_differential FROM high_minute_stats;"
    )
results = cursor.fetchall()
df = pd.DataFrame(results)
df

,average_points,average_differential
0,20.2571,0.0870


Insight - This shows that the high volume offensive players are able to have a positive net differential on average. This is important because with the quality of defense being lowered, as seen in the primary question, by these same players these results tell us that the players have been able to counter this fact by being more careful with the ball on the other end, despite the fact that they are playing the most minutes of any other player.